# Preparation

In [1]:
import os
import cv2
import shutil
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video

In [2]:
DATASET_PATH = "/kaggle/input/snooker-balls/balls"
OUTPUT_PATH = "/kaggle/working"

# Dataset 

### Original Dataset Structure
Dataset structure is given in next format:
```
snooker-balls/balls/
    train/
        {class1}/
            {id_1}.jpg
            {id_2}.jpg
            ...
        {class2}/
            ...
        ...
    test/
    ...
```

### Yolo Dataset Structure
Dataset structure should be transformed to next format:
```
yolo_dataset/
    images/
        train/
            {class1}_{id_1}.jpg
            {class1}_{id_2}.jpg
            ...
        val/
            ...
    labels/
        train/
            {class1}_{id_1}.txt
            {class1}_{id_2}.txt
            ...
        val/
            ...
```

In [3]:
# root directory to save dataset in yolo format
root_dir=os.path.join(OUTPUT_PATH,"yolo_dataset")
os.makedirs(root_dir, exist_ok=True)

# train and test subdirectories with image directory
images_dir=os.path.join(root_dir,"images")
os.makedirs(images_dir, exist_ok=True)
os.makedirs(images_dir+"/train", exist_ok=True)
os.makedirs(images_dir+"/val", exist_ok=True)

# train and test subdirectories with label directory
labels_dir=os.path.join(root_dir,"labels")
os.makedirs(labels_dir, exist_ok=True)
os.makedirs(labels_dir+"/train", exist_ok=True)
os.makedirs(labels_dir+"/val", exist_ok=True)

In [4]:
# variables to convert label/id to id/label
label2id = {"black": 0, "blue": 1, "brown": 2, "green": 3, "pink": 4, "red": 5, "white": 6, "yellow": 7}
id2label = {v: k for k, v in label2id.items()}

In [5]:
for layer1 in os.listdir(DATASET_PATH):
    for layer2 in os.listdir(f'{DATASET_PATH}/{layer1}'):
        for layer3 in os.listdir(f'{DATASET_PATH}/{layer1}/{layer2}'):
            if layer3.endswith('.jpg'):
                shutil.copyfile(f'{DATASET_PATH}/{layer1}/{layer2}/{layer3}',f'{images_dir}/{layer1}/{layer2}_{layer3}'.replace('test', 'val' ))
                f = open(f'{labels_dir}/{layer1}/{layer2}_{layer3}'[:-4].replace('test', 'val' )+'.txt', 'w')
                f.write(str(label2id[layer2])+' 0.5 0.5 1 1')
                f.close()

# YOLOv8

In [6]:
!pip install ultralytics
!yolo checks
from ultralytics import YOLO

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/28/76/3f4ba6c345461cbf79fb45828cdc42b15f60b65a3474e35ad6f476d1d4c7/ultralytics-8.0.230-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.2/663.2 kB 31.5 MB/s eta 0:00:00
Ultralytics YOLOv8.0.230 🚀 Python-3.10.12 torch-2.0.0+cpu CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5310.5/8062.4 GB disk)

OS                  Linux-5.15.133+-x86_64-with-glibc2.31
Environment         Kaggle
Python              3.10.12
Install             pip
RAM                 31.36 GB
CPU                 Intel Xeon 2.20GHz
CUDA                None

matplotlib          ✅ 3.7.4>=3.3.0
numpy               ✅ 1.24.3>=1.22.2
opencv-python       ✅ 4.8.1.78>=4.6.0
pillow              ✅ 9.5.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.11.4>=1.4.1
torch               ✅ 2.0.0+cpu>=1.8.0
torchvision         ✅ 0.15.1+cpu>=0.9.0
t

### Convert Dataset to Yolo Format

In [7]:
names_content = "\n".join([f"  {label_id}: {label}" for label, label_id in label2id.items()])
dataset_content = f"""
path: "{root_dir}/"
train: "images/train"
val: "images/val"
test: "images/val"
names:
{names_content}
"""
with open(os.path.join(OUTPUT_PATH, "custom_dataset.yaml"), "w") as f:
    f.write(dataset_content)

### Train the Model

In [9]:
# pretrained model: yolov8n、yolov8s、yolov8m、yolov8l、yolov8x
model = YOLO('yolov8n.yaml').load('yolov8n.pt')

# Train the model using the processed dataset
results = model.train(
    data=os.path.join(OUTPUT_PATH,'custom_dataset.yaml'),
    project='snooker_project',
    epochs=50,
    batch=64,
    imgsz=128,
    nbs=64,
    val=True,
    optimizer='Adam',
    lr0=0.001,
    lrf=0.0005
)


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

train: Scanning /kaggle/working/yolo_dataset/labels/train.cache... 11510 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11510/11510 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 2873 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2873/2873 [00:00<?, ?it/s]

Plotting labels to snooker_project/train2/labels.jpg... 


...

                   all       2873       2873      0.912      0.996      0.958      0.957



50 epochs completed in 3.364 hours.
Optimizer stripped from snooker_project/train2/weights/last.pt, 6.2MB
Optimizer stripped from snooker_project/train2/weights/best.pt, 6.2MB

Validating snooker_project/train2/weights/best.pt...
Ultralytics YOLOv8.0.230 🚀 Python-3.10.12 torch-2.0.0+cpu CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3007208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:25<00:00,  1.12s/it]


                   all       2873       2873       0.99      0.991      0.994      0.994
                 black       2873        359          1      0.956      0.995      0.995
                  blue       2873        323      0.977      0.991       0.99       0.99
                 brown       2873        331       0.99      0.997      0.995      0.995
                 green       2873        239      0.998          1      0.995      0.995
                  pink       2873         42      0.992          1      0.995      0.995
                   red       2873        322      0.988      0.985      0.989      0.989
                 white       2873       1233          1      0.997      0.995      0.995
                yellow       2873         24      0.974          1      0.995      0.995
Speed: 0.1ms preprocess, 6.7ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to snooker_project/train2


lr/pg0,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▃▆████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▃▆████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▅██████████████████▇████▇██████▇███████
metrics/mAP50-95(B),▁▅█████████▇▇██▇█▇██▇████▇█▇▇███▇▇██████
metrics/precision(B),▁▄▇▆▇█▄▅▄▆▆▇▅▅▆▆▆▆▆█▂▆▆█▆▂▆▆▆▅▅▅▆▆▆▆▆▆▆█
metrics/recall(B),▁▄█▇▇▇▇▇▇████▇██▆▇██▇██▇████████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▆▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁


### Validate the Model

In [10]:
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

Ultralytics YOLOv8.0.230 🚀 Python-3.10.12 torch-2.0.0+cpu CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3007208 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 2873 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2873/2873 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:24<00:00,  1.84it/s]


                   all       2873       2873       0.99      0.991      0.994      0.994
                 black       2873        359          1      0.956      0.995      0.995
                  blue       2873        323      0.977      0.991       0.99       0.99
                 brown       2873        331       0.99      0.997      0.995      0.995
                 green       2873        239      0.998          1      0.995      0.995
                  pink       2873         42      0.992          1      0.995      0.995
                   red       2873        322      0.988      0.985      0.989      0.989
                 white       2873       1233          1      0.997      0.995      0.995
                yellow       2873         24      0.974          1      0.995      0.995
Speed: 0.1ms preprocess, 6.3ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to snooker_project/train22


array([     0.9946,     0.99013,     0.99464,       0.995,       0.995,     0.98905,       0.995,       0.995])

### Export the Model with **ONNX**

In [11]:
model.export(format='onnx')

Ultralytics YOLOv8.0.230 🚀 Python-3.10.12 torch-2.0.0+cpu CPU (Intel Xeon 2.20GHz)

PyTorch: starting from 'snooker_project/train2/weights/best.pt' with input shape (1, 3, 128, 128) BCHW and output shape(s) (1, 12, 336) (5.9 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
============== Diagnostic Run torch.onnx.export version 2.0.0+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: export success ✅ 1.7s, saved as 'snooker_project/train2/weights/best.onnx' (11.5 MB)

Export complete (3.0s)
Results saved to /kaggle/working/snooker_project/train2/weights
Predict:         yolo predict task=detect model=snooker_project/train2/weights/best.onnx imgsz=128  
Validate:        yolo val task=detect model=snooker_project/train2/weights/best.onnx imgsz=128 data=/kaggle/working/custom_dataset.yaml  
Visualize:       https://netron.app


'snooker_project/train2/weights/best.onnx'

### Benchmark the Model with **ONNX**

In [19]:
from ultralytics.utils.benchmarks import benchmark
benchmark(model='snooker_project/train2/weights/best.pt', imgsz=128)

Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5311.5/8062.4 GB disk)

Benchmarks complete for best.pt on coco8.yaml at imgsz=128 (365.37s)
                   Format Status❔  Size (MB)  metrics/mAP50-95(B)  Inference time (ms/im)
0                 PyTorch       ✅        5.9               0.0000                   19.38
1             TorchScript       ✅       11.8               0.0028                  307.26
2                    ONNX       ✅       11.5               0.0028                    8.69
3                OpenVINO       ✅       11.7               0.0028                    9.34
4                TensorRT       ❌        0.0                  NaN                     NaN
5                  CoreML       ❎        5.8                  NaN                     NaN
6   TensorFlow SavedModel       ✅       29.0               0.0028                   72.69
7     TensorFlow GraphDef       ✅       11.6               0.0028                  146.04
8         TensorFlow Lite       ✅       11.5              

,Format,Status❔,Size (MB),metrics/mAP50-95(B),Inference time (ms/im)
0,PyTorch,✅,5.9,0.0000,19.38
1,TorchScript,✅,11.8,0.0028,307.26
2,ONNX,✅,11.5,0.0028,8.69
3,OpenVINO,✅,11.7,0.0028,9.34
4,TensorRT,❌,0.0,NaN,NaN
5,CoreML,❎,5.8,NaN,NaN
6,TensorFlow SavedModel,✅,29.0,0.0028,72.69
7,TensorFlow GraphDef,✅,11.6,0.0028,146.04
8,TensorFlow Lite,✅,11.5,0.0028,7.57
9,TensorFlow Edge TPU,❎,3.2,NaN,NaN


### Create a Download Link

In [ ]:
%cd $OUTPUT_PATH
!zip -r snooker_project.zip snooker_project
from IPython.display import FileLink
FileLink(r'snooker_project.zip')

# TensorBoard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ultralytics/runs

# Video with PyTube

In [13]:
!pip install pytube
from pytube import YouTube
YouTube('https://youtu.be/hw02UKK4Kb0').streams.filter().get_highest_resolution().download(output_path=OUTPUT_PATH, filename='youtube.mp4')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.7 MB/s eta 0:00:00


'/kaggle/working/youtube.mp4'

In [37]:
%cd $OUTPUT_PATH
!ffmpeg -i youtube.mp4 -vcodec copy -acodec copy -ss 00:01:05 -to 00:01:15 video.mp4 -y

/kaggle/working
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enab

In [38]:
%cd $OUTPUT_PATH
Video('video.mp4')

/kaggle/working


# Predict the Video

In [29]:
%cd $OUTPUT_PATH
video_model = YOLO("snooker_project/train2/weights/best.pt")
video_model.predict(source="video.mp4", show=False, save=True)

/kaggle/working


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/301) /kaggle/working/video.mp4: 96x128 1 black, 21.2ms
video 1/1 (2/301) /kaggle/working/video.mp4: 96x128 1 black, 15.0ms
video 1/1 (3/301) /kaggle/working/video.mp4: 96x128 1 black, 14.2ms
video 1/1 (4/301) /kaggle/working/video.mp4: 96x128 1 black, 12.9ms
video 1/1 (5/301) /kaggle/working/video.mp4: 96x128 1 black, 13.3ms
video 1/1 (6/301) /kaggle/working/video.mp4: 96x128 1 black, 13.2ms
video 1/1 (7/301) /kaggle/workin

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'black', 1: 'blue', 2: 'brown', 3: 'green', 4: 'pink', 5: 'red', 6: 'white', 7: 'yellow'}
 orig_img: array([[[ 1,  1,  1],
         [ 1,  1,  1],
         [ 1,  1,  1],
         ...,
         [ 0,  3,  3],
         [ 0,  3,  3],
         [ 0,  3,  3]],
 
        [[ 1,  1,  1],
         [ 1,  1,  1],
         [ 1,  1,  1],
         ...,
         [ 0,  3,  3],
         [ 0,  3,  3],
         [ 0,  3,  3]],
 
        [[ 1,  1,  1],
         [ 1,  1,  1],
         [ 1,  1,  1],
         ...,
         [ 0,  3,  3],
         [ 0,  3,  3],
         [ 0,  3,  3]],
 
        ...,
 
        [[91, 97, 92],
         [91, 97, 92],
         [91, 97, 92],
         ...,
         [91, 97, 92],
         [91, 97, 92],
         [91, 97, 92]],
 
        [[91, 97, 92],
         [91, 97, 92],
         [91, 97, 92],
         ...,
         [91, 97, 92],
       

In [31]:
%cd $OUTPUT_PATH
!ffmpeg -y -loglevel panic -i runs/detect/predict6/video.avi predict_video.mp4
# Display the video 
Video("predict_video.mp4", width=960)

/kaggle/working


# Track the Video with **ByteTrack**

In [18]:
%cd $OUTPUT_PATH
videl_model = YOLO('snooker_project/train2/weights/best.pt')
videl_model.track(source="video.mp4", tracker="bytetrack.yaml", save=True)

/kaggle/working
requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
  Obtaining dependency information for lapx>=0.5.2 from https://files.pythonhosted.org/packages/b1/1c/1fd887004a4d68b9694e5cfe38cf450758dcf7d7436fbe576f7a5b9293c2/lapx-0.5.5-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 11.5s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'black', 1: 'blue', 2: 'brown', 3: 'green', 4: 'pink', 5: 'red', 6: 'white', 7: 'yellow'}
 orig_img: array([[[ 3,  3,  3],
         [ 3,  3,  3],
         [ 2,  2,  2],
         ...,
         [ 3,  3,  1],
         [ 3,  3,  1],
         [ 3,  3,  1]],
 
        [[ 3,  3,  3],
         [ 3,  3,  3],
         [ 2,  2,  2],
         ...,
         [ 3,  3,  1],
         [ 3,  3,  1],
         [ 3,  3,  1]],
 
        [[ 4,  4,  4],
         [ 4,  4,  4],
         [ 3,  3,  3],
         ...,
         [ 3,  3,  1],
         [ 3,  3,  1],
         [ 3,  3,  1]],
 
        ...,
 
        [[91, 97, 92],
         [91, 97, 92],
         [91, 97, 92],
         ...,
         [91, 97, 92],
         [91, 97, 92],
         [91, 97, 92]],
 
        [[91, 97, 92],
         [91, 97, 92],
         [91, 97, 92],
         ...,
         [91, 97, 92],
       

In [22]:
%cd $OUTPUT_PATH
!ffmpeg -y -loglevel panic -i runs/detect/predict2/video.avi track_video.mp4
# Display the video 
Video("track_video.mp4", width=960)

/kaggle/working
